<a href="https://colab.research.google.com/github/58191554/PointNet-Project/blob/TongZhen-branch/pointNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install torch matplotlib numpy os
import sys
import os
print("\nPython version")
print(sys.version)
print("Python version info")
print(sys.version_info)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os

Python version
3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0]
Python version info
sys.version_info(major=3, minor=9, micro=16, releaselevel='final', serial=0)


In [37]:
from google.colab import drive
drive.mount('/content/drive')
root_folder = "/content/drive/MyDrive/PointNet/"

os.makedirs(root_folder, exist_ok=True)
os.chdir(root_folder)

import zipfile
import urllib.request

if not os.path.exists("ModelNet10.zip"):
    print("Down Loading ModelNet10.zip ...")
    urllib.request.urlretrieve("https://vision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip", "ModelNet10.zip")
    print("Finish DownLoading！")

if not os.path.exists("ModelNet10"):
    print("unzipping ModelNet10.zip...")
    with zipfile.ZipFile("ModelNet10.zip", "r") as zip_ref:
        zip_ref.extractall(".")
    print("Finish unzipping")
else:
    print("ModelNet10 Exists")

def remove_ds_store(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file == ".DS_Store":
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print("Deleted:", file_path)

remove_ds_store(root_folder+"ModelNet10")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ModelNet10 Exists


In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm
import random

In [39]:
class_str_list = [
    "bathtub",
    "bed",
    "chair",
    "desk",
    "dresser",
    "monitor",
    "night_stand",
    "sofa",
    "table",
    "toilet"
]

class_to_idx = {}
idx_to_class = {}
for idx, class_str in enumerate(class_str_list):
  class_to_idx[class_str] = idx
  idx_to_class[idx] = class_str

for key, value in class_to_idx.items():
    print(f"key：{key}, value：{value}")

key：bathtub, value：0
key：bed, value：1
key：chair, value：2
key：desk, value：3
key：dresser, value：4
key：monitor, value：5
key：night_stand, value：6
key：sofa, value：7
key：table, value：8
key：toilet, value：9


In [40]:
class PointDataSet(torch.utils.data.Dataset):
  def __init__(self, root_dir):
    self.root_dir = root_dir
    self.items = []      #N
    self.classes = []     #[0, 1, 2,...,10]
    self.class_to_idx = class_to_idx
    self.idx_to_class = idx_to_class
    self.data = []       #NxMx3
    self.label = []      #N
    self.y = None       #Nx10

    folders = os.listdir(root_dir)
        
    # get different objects
    for idx, item_folder in idx_to_class.items():

      # if idx > 2:break
      print(idx, item_folder)
      if not os.path.isdir(os.path.join(root_dir, item_folder)):
        continue
            
      # record class
      self.classes.append(item_folder)
      self.class_to_idx[item_folder] = idx
      self.idx_to_class[idx] = item_folder
            
      # get train data
      train_folder = os.path.join(root_dir, item_folder, 'train')
      if os.path.exists(train_folder):
        train_files = [f for f in os.listdir(train_folder) if f.endswith('.off')]
        self.items.extend([(os.path.join(train_folder, f), idx) for f in train_files])
        self.label.extend([idx for f in train_files])

            
      # get test data
      test_folder = os.path.join(root_dir, item_folder, 'test')
      if os.path.exists(test_folder):
        test_files = [f for f in os.listdir(test_folder) if f.endswith('.off')]
        self.items.extend([(os.path.join(test_folder, f), idx) for f in test_files])
        self.label.extend([idx for f in test_files])

      # break

    # get tensor data
    self.load_to_RAM()

  def __getitem__(self, index):
      return self.data[index], self.y[index]

  def __len__(self):
      return len(self.items)

  def load_to_RAM(self):
    # reduce I/O, read into train/test_data list of torch tensor
    print("Load classes: ", self.classes)
    self.y = torch.zeros((len(self.items),len(self.classes)))
    for i, (f, c) in tqdm(enumerate(self.items), dynamic_ncols=True, leave=True):
      points = self.read_off_file(f)
      self.data.append(torch.from_numpy(points))
      self.y[i,c] = 1 
    
  def read_off_file(self, file_path):
    # return numpy
    with open(file_path, 'r') as f:
      lines = f.readlines()
      num_points = int(lines[1].split()[0])
      points = np.zeros((num_points, 3), dtype=np.float32)
      for i in range(num_points):
        point = lines[i + 2].split()
        points[i] = [float(point[0]), float(point[1]), float(point[2])]
    return points

  def get_min_points_num(self):
    # return the minimum number of points among all the objects.
    min_num = self[0][0].size(0)
    for ts in self.data:
      if(min_num > ts.size(0)):
        min_num = ts.size(0)
    return min_num

  def get_unify_data(self):
    min_num = self.get_min_points_num()
    unify_data = []
    unify_data.extend([random_sample(points, min_num) for points in self.data])
    return unify_data

def plot3d(points, plot_line = False):
  fig = plt.figure()
  ax = fig.add_subplot(111, projection='3d')

  # get xyz coordinate data
  x = points[:, 0]
  y = points[:, 1]
  z = points[:, 2]

  # plot vertices
  ax.scatter(x, y, z, c='b', marker='o', s=1)  # c: 点的颜色，marker: 点的形状，s: 点的大小

  # plot line
  if plot_line:
    for i in range(points.shape[0]):
      ax.plot([x[i], x[(i + 1) % points.shape[0]]],
          [y[i], y[(i + 1) % points.shape[0]]],
          [z[i], z[(i + 1) % points.shape[0]]], c='r', linewidth=0.5)

  # set axis label
  ax.set_xlabel('X')
  ax.set_ylabel('Y')
  ax.set_zlabel('Z')

  plt.show()

def random_sample(points, num):
  if num > points.size(0):
    return []
  indices = torch.randperm(points.size(0))[:num]
  # 从输入的 torch tensor 中根据索引选择对应的向量
  selected_vectors = [points[i] for i in indices]

  sample = torch.cat(selected_vectors, dim=0)
  return sample

In [41]:
data_path = "/content/drive/MyDrive/PointNet/ModelNet10"
pointData = PointDataSet(data_path)
points, label = pointData[2]
# plot3d(points)

0 bathtub
1 bed
2 chair
3 desk
4 dresser
5 monitor
6 night_stand
7 sofa
8 table
9 toilet
Load classes:  ['bathtub', 'bed', 'chair', 'desk', 'dresser', 'monitor', 'night_stand', 'sofa', 'table', 'toilet']


4899it [11:42,  6.97it/s]


In [42]:
for i in range(len(pointData)):
  print(pointData[i][1])

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 

In [43]:
# 生成规定长度的数据集，用于MLP
class UnifyDataset(torch.utils.data.Dataset):
  def __init__(self, x, y):
    self.x = x
    self.y = y
        
  def __getitem__(self, index):
    x_item = self.x[index]
    y_item = self.y[index]
    return x_item, y_item
    
  def __len__(self):
    return len(self.x)

Try MLP

In [44]:
train_ratio = 0.8
test_ratio = 0.2
batch_size = 16

train_size = int(train_ratio * len(pointData))
test_size = len(pointData) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(pointData, [train_size, test_size])

# 创建训练数据和测试数据
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

uniData = UnifyDataset(pointData.get_unify_data(), pointData.label)
print(len(uniData))
train_unify_dataset, test_unify_dataset= torch.utils.data.random_split(uniData, [train_size, test_size])
train_unify_dataloader = torch.utils.data.DataLoader(train_unify_dataset, batch_size=batch_size, shuffle=True)
test_unify_dataloader = torch.utils.data.DataLoader(test_unify_dataset, batch_size=batch_size, shuffle=False)

4899


In [45]:
class MLP(nn.Module):
  def __init__(self, layers):
    super(MLP, self).__init__()
    self.layers = nn.ModuleList()  # 使用 ModuleList 存储每一层的 Module
    for i in range(1, len(layers)):
      self.layers.append(nn.Linear(layers[i-1], layers[i]))
    
  def forward(self, x):
    for layer in self.layers:
      x = torch.relu(layer(x))
      x = torch.softmax(x, dim=0)
    return x

Train MLP 

In [46]:
min_num_point = pointData.get_min_points_num()
print(min_num_point)
layers = [min_num_point * 3, 64, 32, 10]  # MLP 模型的每层维度
model = MLP(layers)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 模型训练
num_epochs = 100  # 设定训练的轮数
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 检查是否有可用的 GPU
model.to(device)  # 将模型移动到对应的设备

for epoch in range(num_epochs):
  model.train()  # 将模型设置为训练模式
  for i, (inputs, targets) in enumerate(train_unify_dataloader):
    inputs, targets = inputs.to(device), targets.to(device)  # 将输入和标签移动到对应的设备
    optimizer.zero_grad()  # 清零优化器梯度
        
    # 前向传播和计算损失
    outputs = model(inputs)
    loss = criterion(outputs, targets)
        
    # 反向传播和优化
    loss.backward()
    optimizer.step()
    
    # 打印训练信息
  if epoch % 10 == 0:
    print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, i + 1, len(train_unify_dataloader), loss.item()))


100
Epoch [1/100], Step [245/245], Loss: 2.3024
Epoch [11/100], Step [245/245], Loss: 2.2700
Epoch [21/100], Step [245/245], Loss: 2.2111
Epoch [31/100], Step [245/245], Loss: 2.2776
Epoch [41/100], Step [245/245], Loss: 2.2402
Epoch [51/100], Step [245/245], Loss: 2.2640
Epoch [61/100], Step [245/245], Loss: 2.2180
Epoch [71/100], Step [245/245], Loss: 2.2444
Epoch [81/100], Step [245/245], Loss: 2.2200
Epoch [91/100], Step [245/245], Loss: 2.2543
